In [1]:
import cv2
import os
import glob
import numpy as np

image_path = "../../dataset/train/images/clip_one/*"

image_files = sorted(glob.glob(image_path))

if not image_files:
    print("No images found in the specified directory.")
else:
    cv2.namedWindow("Video", cv2.WINDOW_NORMAL)

    backSub = cv2.createBackgroundSubtractorMOG2(history=0, varThreshold=1000)
    backSub.setBackgroundRatio(0.9)

    prev_gray = None
    thresh = 5

    for img_file in image_files:
        frame = cv2.imread(img_file)
        if frame is None:
            print(f"Could not read image: {img_file}")
            continue

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        _, binary_frame = cv2.threshold(gray_frame, 150, 255, cv2.THRESH_TOZERO)
        fg_mask = backSub.apply(binary_frame)

        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_DILATE, kernel)

        fg_mask = cv2.GaussianBlur(fg_mask, (5, 5), 2)

        if prev_gray is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_gray, gray_frame, None, 0.5, 3, 5, 3, 5, 1.2, 0)
            mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            motion_mask = (mag > thresh).astype(np.uint8) * 255
            filtered_frame = cv2.bitwise_and(frame, frame, mask=motion_mask)
            cv2.imshow("Video", filtered_frame)
            cv2.imshow("fg_mask", fg_mask)
            cv2.imshow("gray", gray_frame)

        prev_gray = gray_frame

        if cv2.waitKey(33) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
